In [1]:
import torch
import librosa

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
file_path = r"N:\Music\Audio Cassettes 1 to 5\cassette 4 side a.mp3"

In [3]:
speech, sr = librosa.load(file_path, sr=16000)
sr, speech.shapeds

c:\Users\lukeasargen\anaconda3\envs\pt111\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


(16000, (1536768,))

In [4]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = "facebook/wav2vec2-base-960h" # 360MB
# model_name = "facebook/wav2vec2-large-960h-lv60-self" # 1.18GB
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# model = model.to(device)
# model = model.eval()

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
input_values = processor(speech, return_tensors="pt", sampling_rate=16000)["input_values"]
input_values.shape

torch.Size([1, 1536768])

In [11]:
logits = model(input_values.to(device)).logits
logits.shape
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])
transcription.lower()


RuntimeError: CUDA out of memory. Tried to allocate 1.03 GiB (GPU 0; 8.00 GiB total capacity; 5.69 GiB already allocated; 0 bytes free; 6.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
from transformers import pipeline
p = pipeline("automatic-speech-recognition", device=0)


No model was supplied, defaulted to facebook/wav2vec2-base-960h (https://huggingface.co/facebook/wav2vec2-base-960h)
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
prediction = p(speech, chunk_length_s=20, stride_length_s=4)
prediction

{'text': "A A A A AA  BE MALL ER BOIAL GOT ON  WOOD ARE WE GNE WHERE WE GONE I SOL BE CORT THE CARS GOING BY WONT SHE HOUT THAT WIRL THAT WORD NOW W'LL TRY WISH AT MEMANDE DECORDING IN THE NORMAL OR LOW MOD WITH T HE MICRO BONE TO PICK UP EVERYTHING IT SEEME TO BE JUST AS SENT OF IT BUT THE LOW MOMMAN WITH THE MIT OF BAN SHEE HOW WELL THE CORN THAT'S KILL BOT NOT TALKING VERY LOUD TALKING LOUD AT ALL SEEMS LIKE THE BEST WAY MAY BE  USE TA BRACK ROUND IN A LOW WOI ATWAIT RECORDING THEM"}